In [162]:
import pandas as pd
import sys
import numpy as np
from sklearn.ensemble import GradientBoostingRegressor, RandomForestRegressor
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error
import math
from math import sqrt
sys.path.insert(0,"../")

In [163]:
import logging

from tqdm.notebook import tqdm
tqdm.pandas()

logger = logging.getLogger(__name__)

logger.setLevel(logging.CRITICAL)

import warnings; warnings.simplefilter('ignore')


# Read in predictions data + true close 

In [164]:
btc_predictions_df = pd.read_csv("training_configs/btc_all_predictions.csv")
true_price_df = pd.read_csv("../tmp/historic_crypto_prices - bitcoin_jan_2017_sep_4_2021 copy.csv")

In [165]:
btc_predictions_df.head()

,date,nbeats_btc_lookback_15_window_5_std_1.25_num_add_dfs_2,tcn_btc_lookback_15_window_5_std_1.25_num_add_dfs_2,nbeats_btc_lookback_30_window_5_std_1.25_num_add_dfs_2,tcn_btc_lookback_30_window_5_std_1.25_num_add_dfs_2,nbeats_btc_lookback_45_window_5_std_1.25_num_add_dfs_2,tcn_btc_lookback_45_window_5_std_1.25_num_add_dfs_2,date_prediction_for,test_model_lookback_1
0,2010-01-01,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,2010-01-09,0.0
1,2019-04-02,4752.149527,253.537135,4730.976374,654.310204,3469.734359,795.201430,2019-04-09,0.0
2,2019-04-03,4546.248123,407.206977,4373.560445,255.627627,4437.900633,1413.509939,2019-04-10,0.0
3,2019-04-04,4550.998590,726.168584,4045.591506,1861.981408,4012.011230,523.093010,2019-04-11,0.0
4,2019-04-05,5003.264197,353.606543,4819.668698,817.215937,3977.238773,553.057301,2019-04-12,0.0


In [166]:
btc_predictions_df.tail()

,date,nbeats_btc_lookback_15_window_5_std_1.25_num_add_dfs_2,tcn_btc_lookback_15_window_5_std_1.25_num_add_dfs_2,nbeats_btc_lookback_30_window_5_std_1.25_num_add_dfs_2,tcn_btc_lookback_30_window_5_std_1.25_num_add_dfs_2,nbeats_btc_lookback_45_window_5_std_1.25_num_add_dfs_2,tcn_btc_lookback_45_window_5_std_1.25_num_add_dfs_2,date_prediction_for,test_model_lookback_1
967,2021-11-23,55918.355465,60103.816830,60360.595433,55696.413167,61206.097441,54648.630211,2021-11-30,0.0
968,2021-11-24,62171.488175,56868.770797,57699.929206,55804.118764,62313.001606,60208.644347,2021-12-01,0.0
969,2021-11-25,56654.252820,56390.781091,60350.362835,59965.168783,53821.808193,57760.215269,2021-12-02,0.0
970,2021-11-26,55409.967586,54564.064367,55769.419866,55586.696260,62471.016254,55356.446759,2021-12-03,0.0
971,2021-11-27,58178.504682,50895.158473,60944.803601,55055.286244,64831.556432,52358.904067,2021-12-04,0.0


In [167]:
true_price_df.head()

,date,open,high,low,close,volume
0,2017-01-01,963.66,1003.08,958.70,998.33,147775008.0
1,2017-01-02,998.62,1031.39,996.70,1021.75,222184992.0
2,2017-01-03,1021.60,1044.08,1021.60,1043.84,185168000.0
3,2017-01-04,1044.40,1159.42,1044.40,1154.73,344945984.0
4,2017-01-05,1156.73,1191.10,910.42,1013.38,510199008.0


- merged true close price and prediction

In [168]:
true_price_df = true_price_df[['date','close']]
merged_df = pd.merge(btc_predictions_df, true_price_df, left_on='date_prediction_for',
                     right_on ='date', suffixes=['_pred','_true'])

In [169]:
merged_df['day'] = [t.day for t in pd.to_datetime(merged_df.date_prediction_for)]
merged_df['month'] =  [t.month for t in pd.to_datetime(merged_df.date_prediction_for)]
merged_df['quarter'] = [t.quarter for t in pd.to_datetime(merged_df.date_prediction_for)]
merged_df['day_of_year'] = [t.strftime('%j') for t in pd.to_datetime(merged_df.date_prediction_for)]

In [170]:
dropped_merged_df = merged_df.drop(['date_pred', 'date_true', 'date_prediction_for'],  axis=1)

# Test the different algos
- mean
- RF
- GB
- LR

In [172]:
dropped_merged_df

,nbeats_btc_lookback_15_window_5_std_1.25_num_add_dfs_2,tcn_btc_lookback_15_window_5_std_1.25_num_add_dfs_2,nbeats_btc_lookback_30_window_5_std_1.25_num_add_dfs_2,tcn_btc_lookback_30_window_5_std_1.25_num_add_dfs_2,nbeats_btc_lookback_45_window_5_std_1.25_num_add_dfs_2,tcn_btc_lookback_45_window_5_std_1.25_num_add_dfs_2,test_model_lookback_1,close,day,month,quarter,day_of_year
0,4752.149527,253.537135,4730.976374,654.310204,3469.734359,795.201430,0.0,5204.96,9,4,2,099
1,4546.248123,407.206977,4373.560445,255.627627,4437.900633,1413.509939,0.0,5324.55,10,4,2,100
2,4550.998590,726.168584,4045.591506,1861.981408,4012.011230,523.093010,0.0,5064.49,11,4,2,101
3,5003.264197,353.606543,4819.668698,817.215937,3977.238773,553.057301,0.0,5089.54,12,4,2,102
4,5184.268504,197.768508,5123.185573,1596.333494,4093.602236,920.106602,0.0,5096.59,13,4,2,103
...,...,...,...,...,...,...,...,...,...,...,...,...
966,55918.355465,60103.816830,60360.595433,55696.413167,61206.097441,54648.630211,0.0,57003.00,30,11,4,334
967,62171.488175,56868.770797,57699.929206,55804.118764,62313.001606,60208.644347,0.0,57254.00,1,12,4,335
968,56654.252820,56390.781091,60350.362835,59965.168783,53821.808193,57760.215269,0.0,56524.00,2,12,4,336
969,55409.967586,54564.064367,55769.419866,55586.696260,62471.016254,55356.446759,0.0,53632.00,3,12,4,337


In [173]:
PRED_COL = 'close'
window = 5
no_of_std = 1.25
stop_loss_pct=0.10
rolling_mean_col = 'rolling_mean'
bollinger_high_col = 'bollinger_high'
bollinger_low_col = 'bollinger_low'

COLS_TO_EXCLUDE = ['test_model_lookback_1', 'close','rolling_mean','bollinger_high','bollinger_low']
DAY_PART_COLS = ['day', 'month', 'quarter', 'day_of_year']


rolling_mean = dropped_merged_df[PRED_COL].rolling(window).mean()
rolling_std = dropped_merged_df[PRED_COL].rolling(window).std()

dropped_merged_df[rolling_mean_col] = rolling_mean
dropped_merged_df[bollinger_high_col] = rolling_mean + (rolling_std * no_of_std)
dropped_merged_df[bollinger_low_col] = rolling_mean - (rolling_std * no_of_std)

In [174]:
dropped_merged_df.dropna(inplace=True)

In [175]:
dropped_merged_df

,nbeats_btc_lookback_15_window_5_std_1.25_num_add_dfs_2,tcn_btc_lookback_15_window_5_std_1.25_num_add_dfs_2,nbeats_btc_lookback_30_window_5_std_1.25_num_add_dfs_2,tcn_btc_lookback_30_window_5_std_1.25_num_add_dfs_2,nbeats_btc_lookback_45_window_5_std_1.25_num_add_dfs_2,tcn_btc_lookback_45_window_5_std_1.25_num_add_dfs_2,test_model_lookback_1,close,day,month,quarter,day_of_year,rolling_mean,bollinger_high,bollinger_low
4,5184.268504,197.768508,5123.185573,1596.333494,4093.602236,920.106602,0.0,5096.59,13,4,2,103,5156.026,5291.705339,5020.346661
5,5217.155644,548.560523,4793.360150,1613.115274,4025.372554,131.548921,0.0,5167.72,14,4,2,104,5148.578,5280.557506,5016.598494
6,5667.192572,1704.822989,5254.864495,1489.430029,4594.841162,211.985893,0.0,5067.11,15,4,2,105,5097.090,5149.408921,5044.771079
7,5527.824658,179.438758,4803.556949,858.146121,4475.389571,519.848001,0.0,5235.56,16,4,2,106,5131.304,5218.054364,5044.553636
8,5415.966694,2598.145400,3666.543647,2173.548003,5277.090149,1192.791923,0.0,5251.94,17,4,2,107,5163.784,5266.100265,5061.467735
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
966,55918.355465,60103.816830,60360.595433,55696.413167,61206.097441,54648.630211,0.0,57003.00,30,11,4,334,56132.800,58369.342113,53896.257887
967,62171.488175,56868.770797,57699.929206,55804.118764,62313.001606,60208.644347,0.0,57254.00,1,12,4,335,56841.400,58331.842625,55350.957375
968,56654.252820,56390.781091,60350.362835,59965.168783,53821.808193,57760.215269,0.0,56524.00,2,12,4,336,57190.600,57790.161428,56591.038572
969,55409.967586,54564.064367,55769.419866,55586.696260,62471.016254,55356.446759,0.0,53632.00,3,12,4,337,56449.600,58505.180172,54394.019828


### determine trading state

In [176]:
mode = 'no_position'
has_crossed_mean = False
buy_entry_price = 0
stop_loss_price = 0


In [177]:
row_idx

827

In [178]:
dropped_merged_df[stacking_training_cols].iloc[2]

nbeats_btc_lookback_15_window_5_std_1.25_num_add_dfs_2    5667.192572
tcn_btc_lookback_15_window_5_std_1.25_num_add_dfs_2       1704.822989
nbeats_btc_lookback_30_window_5_std_1.25_num_add_dfs_2    5254.864495
tcn_btc_lookback_30_window_5_std_1.25_num_add_dfs_2       1489.430029
nbeats_btc_lookback_45_window_5_std_1.25_num_add_dfs_2    4594.841162
tcn_btc_lookback_45_window_5_std_1.25_num_add_dfs_2        211.985893
day                                                                15
month                                                               4
quarter                                                             2
day_of_year                                                       105
Name: 6, dtype: object

In [179]:
gb_pred_list = []
rf_pred_list = []
lr_pred_list = []
mean_pred_list = []
actual_close_list = []

for row_idx, row in enumerate(dropped_merged_df.iterrows()):
    if row_idx == 0: # need one day to start training
        continue
    
    if row_idx % 10 == 0:
        print(f"{row_idx/len(dropped_merged_df)*100:.2f}%")

    gb = GradientBoostingRegressor(n_estimators=150)
    rf = RandomForestRegressor(n_estimators=200, n_jobs=-1)
    lr = LinearRegression()
    
    
    stacking_training_cols = row[1].index[~row[1].index.isin(COLS_TO_EXCLUDE)]
    stacked_x_data = dropped_merged_df[stacking_training_cols][:row_idx] # don't include the current row
    stacked_y_data = dropped_merged_df[PRED_COL][:row_idx] # don't include the current row
    
    gb.fit(stacked_x_data, stacked_y_data)
    gb_pred = gb.predict(np.array(dropped_merged_df[stacking_training_cols].iloc[row_idx]).reshape(1,-1))
    gb_pred_list.append(gb_pred[0])
    
    rf.fit(stacked_x_data, stacked_y_data)
    rf_pred = rf.predict(np.array(dropped_merged_df[stacking_training_cols].iloc[row_idx]).reshape(1,-1))
    rf_pred_list.append(rf_pred[0])
    
    lr.fit(stacked_x_data, stacked_y_data)
    lr_pred = lr.predict(np.array(dropped_merged_df[stacking_training_cols].iloc[row_idx]).reshape(1,-1))
    lr_pred_list.append(lr_pred[0])
    
    
    mean_pred_cols = row[1].index[~row[1].index.isin(COLS_TO_EXCLUDE+DAY_PART_COLS)]
    mean_pred = np.mean(row[1][mean_pred_cols])
    mean_pred_list.append(mean_pred)
    
    actual_close_list.append(row[1][PRED_COL])


1.03%
2.07%
3.10%
4.14%
5.17%
6.20%
7.24%
8.27%
9.31%
10.34%
11.38%
12.41%
13.44%
14.48%
15.51%
16.55%
17.58%
18.61%
19.65%
20.68%
21.72%
22.75%
23.78%
24.82%
25.85%
26.89%
27.92%
28.96%
29.99%
31.02%
32.06%
33.09%
34.13%
35.16%
36.19%
37.23%
38.26%
39.30%
40.33%
41.37%
42.40%
43.43%
44.47%
45.50%
46.54%
47.57%
48.60%
49.64%
50.67%
51.71%
52.74%
53.77%
54.81%
55.84%
56.88%
57.91%
58.95%
59.98%
61.01%
62.05%
63.08%
64.12%
65.15%
66.18%
67.22%
68.25%
69.29%
70.32%
71.35%
72.39%
73.42%
74.46%
75.49%
76.53%
77.56%
78.59%
79.63%
80.66%
81.70%
82.73%
83.76%
84.80%
85.83%
86.87%
87.90%
88.93%
89.97%
91.00%
92.04%
93.07%
94.11%
95.14%
96.17%
97.21%
98.24%
99.28%


In [180]:
rf_pred_list[-10:]

[59357.9,
 59205.25,
 56941.21,
 56362.04815,
 56348.355,
 56840.06,
 56410.3445,
 56963.0814,
 56560.08784999999,
 56224.41735]

In [181]:
mean_pred_list[-10:]

[60239.696520741214,
 58163.26547876602,
 58194.001785175824,
 58628.04618061924,
 57066.05090490591,
 57988.98475793731,
 59177.65881577427,
 57490.4314984547,
 56526.2685154095,
 57044.035583296325]

In [182]:
gb_pred_list[-10:]

[59114.00863856613,
 58193.32319445432,
 57343.255520973864,
 57614.63554273005,
 55911.818168019265,
 56810.03163419688,
 59378.55568322487,
 59165.24350095437,
 58778.22929636356,
 58559.329370876854]

In [183]:
rf_pred_list[-10:]

[59357.9,
 59205.25,
 56941.21,
 56362.04815,
 56348.355,
 56840.06,
 56410.3445,
 56963.0814,
 56560.08784999999,
 56224.41735]

In [184]:
actual_close_list[-10:]

[58982.0,
 53711.0,
 54778.0,
 57337.0,
 57835.0,
 57003.0,
 57254.0,
 56524.0,
 53632.0,
 49232.0]

In [193]:
dropped_merged_df["nbeats_btc_lookback_15_window_5_std_1.25_num_add_dfs_2"]

4       5184.268504
5       5217.155644
6       5667.192572
7       5527.824658
8       5415.966694
           ...     
966    55918.355465
967    62171.488175
968    56654.252820
969    55409.967586
970    58178.504682
Name: nbeats_btc_lookback_15_window_5_std_1.25_num_add_dfs_2, Length: 967, dtype: float64

In [204]:
rmse_nbeats_15 = sqrt(mean_squared_error(dropped_merged_df["nbeats_btc_lookback_15_window_5_std_1.25_num_add_dfs_2"], dropped_merged_df.close))
rmse_nbeats_30 = sqrt(mean_squared_error(dropped_merged_df["nbeats_btc_lookback_30_window_5_std_1.25_num_add_dfs_2"], dropped_merged_df.close))
rmse_nbeats_45 = sqrt(mean_squared_error(dropped_merged_df["nbeats_btc_lookback_45_window_5_std_1.25_num_add_dfs_2"], dropped_merged_df.close))

rmse_tcn_15 = sqrt(mean_squared_error(dropped_merged_df["tcn_btc_lookback_15_window_5_std_1.25_num_add_dfs_2"], dropped_merged_df.close))
rmse_tcn_30 = sqrt(mean_squared_error(dropped_merged_df["tcn_btc_lookback_30_window_5_std_1.25_num_add_dfs_2"], dropped_merged_df.close))
rmse_tcn_45 = sqrt(mean_squared_error(dropped_merged_df["tcn_btc_lookback_45_window_5_std_1.25_num_add_dfs_2"], dropped_merged_df.close))

In [205]:
rmse_gb = sqrt(mean_squared_error(gb_pred_list, actual_close_list))
rmse_rf = sqrt(mean_squared_error(rf_pred_list, actual_close_list))
rmse_lr = sqrt(mean_squared_error(lr_pred_list, actual_close_list))
rmse_average = sqrt(mean_squared_error(mean_pred_list, actual_close_list))

print(f"rmse_gb = {rmse_gb}")
print(f"rmse_rf = {rmse_rf}")
print(f"rmse_lr = {rmse_lr}")
print(f"rmse_average = {rmse_average}")
print(f" rmse_nbeats_15 rmse ={rmse_nbeats_15}")
print(f" rmse_nbeats_30 rmse ={rmse_nbeats_30}")
print(f" rmse_nbeats_45 rmse ={rmse_nbeats_45}")
print(f" rmse_tcn_15 rmse ={rmse_tcn_15}")
print(f" rmse_tcn_30 rmse ={rmse_tcn_30}")
print(f" rmse_tcn_45 rmse ={rmse_tcn_45}")

rmse_gb = 1952.886425557461
rmse_rf = 1851.738028049921
rmse_lr = 3745.8616879919196
rmse_average = 3891.467857825904
 rmse_nbeats_15 rmse =4143.872094745935
 rmse_nbeats_30 rmse =4410.835007346497
 rmse_nbeats_45 rmse =4798.191278557834
 rmse_tcn_15 rmse =4514.463941189475
 rmse_tcn_30 rmse =4508.77412356865
 rmse_tcn_45 rmse =4840.856072099396


In [186]:
rmse_rf/rmse_gb-1 # 4% better

-0.051794306204297924

In [187]:
rmse_rf/rmse_average-1 # lol 56% better

-0.5241543562216507